# Without trusted aggregator

In [2]:
import torch as th
import syft as sy

ModuleNotFoundError: ignored

In [0]:
hook = sy.TorchHook(th)

In [0]:
naufil = sy.VirtualWorker(hook,id="naufil")
faizan = sy.VirtualWorker(hook,id="faizan")

In [0]:
naufil.add_workers([faizan])
faizan.add_workers([naufil])

In [0]:
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]],requires_grad=True)
target = th.tensor([[1.],[1],[0],[0]],requires_grad = True)

In [0]:
naufil_data = data[0:2].send(naufil)
naufil_target = target[0:2].send(naufil)
faizan_data = data[2:4].send(faizan)
faizan_target = target[2:4].send(faizan)

In [0]:
dataset = [(naufil_data,naufil_target),(faizan_data,faizan_target)]

In [0]:
from torch import nn,optim
model = nn.Linear(2,1)
opt = optim.SGD(params = model.parameters(), lr = 0.1)

In [0]:
def train(iterations=20):
  for i in range(iterations):
    for images,targets in dataset:
      opt.zero_grad()
      worker = images.location
      model.send(worker)
      pred = model(images)
      loss = ((pred - targets)**2).sum()
      loss.backward()
      opt.step()
      model.get()

      print(loss.get())



In [0]:
train()

In [0]:
model(data[0])

# A trusted aggregator

In [7]:
# We gotta make separate models for each virtual worker
!pip install syft

     |████████████████████████████████| 256kB 2.8MB/s 
     |████████████████████████████████| 460kB 40.3MB/s 
     |████████████████████████████████| 389kB 35.5MB/s 
     |████████████████████████████████| 256kB 42.3MB/s 
     |████████████████████████████████| 204kB 43.9MB/s 
     |████████████████████████████████| 1.4MB 41.5MB/s 
     |████████████████████████████████| 81kB 21.5MB/s 
     |████████████████████████████████| 51kB 12.4MB/s 
     |████████████████████████████████| 266kB 39.7MB/s 
     |████████████████████████████████| 122kB 41.7MB/s 
  Created wheel for zstd: filename=zstd-1.4.1.0-cp36-cp36m-linux_x86_64.whl size=1067085 sha256=a4fb2060309052482daf1d9880e2e62d08712ce5188370d1a63b08d54abdb7e5
  Stored in directory: /root/.cache/pip/wheels/66/3f/ee/ac08c81af7c1b24a80c746df669ea3cb37542d27877d66ccf4
  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp36-cp36m-linux_x86_64.whl size=44105 sha256=4d302f7ff9094015793cd47790b1fdc3714b1727a3ba7e42c3671cb6dd3bef12
  Stored in di

In [8]:
import torch as th
from torch import nn,optim
import syft as sy

W0811 10:32:57.517310 139656388671360 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0811 10:32:57.536475 139656388671360 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
hook = sy.TorchHook(th)
naufil = sy.VirtualWorker(hook,id="naufil")
faizan = sy.VirtualWorker(hook,id="faizan")
trusted_aggregator = sy.VirtualWorker(hook,id="trusted_aggregator")

In [10]:
naufil.add_workers([faizan,trusted_aggregator])
faizan.add_workers([naufil,trusted_aggregator])
trusted_aggregator.add_workers([naufil,faizan])

W0811 10:33:04.152960 139656388671360 base.py:646] Worker faizan already exists. Replacing old worker which could cause                     unexpected behavior
W0811 10:33:04.154679 139656388671360 base.py:646] Worker trusted_aggregator already exists. Replacing old worker which could cause                     unexpected behavior
W0811 10:33:04.155907 139656388671360 base.py:646] Worker naufil already exists. Replacing old worker which could cause                     unexpected behavior
W0811 10:33:04.157330 139656388671360 base.py:646] Worker trusted_aggregator already exists. Replacing old worker which could cause                     unexpected behavior
W0811 10:33:04.158828 139656388671360 base.py:646] Worker naufil already exists. Replacing old worker which could cause                     unexpected behavior
W0811 10:33:04.162036 139656388671360 base.py:646] Worker faizan already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:trusted_aggregator #objects:0>

In [0]:
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]],requires_grad=True)
target = th.tensor([[1.],[1],[0],[0]],requires_grad = True)

In [0]:
naufil_data = data[0:2].send(naufil)
naufil_target = target[0:2].send(naufil)
faizan_data = data[2:4].send(faizan)
faizan_target = target[2:4].send(faizan)

In [0]:
model = nn.Linear(2,1)

In [14]:
for round_iter in range(20):
  naufil_model = model.copy().send(naufil)
  faizan_model = model.copy().send(faizan)

  naufil_optimizer = optim.SGD(params=naufil_model.parameters(),lr = 0.1)
  faizan_optimizer = optim.SGD(params=faizan_model.parameters(),lr = 0.1)

  for i in range(10):
    naufil_optimizer.zero_grad()
    naufil_pred = naufil_model(naufil_data)
    naufil_loss = ((naufil_pred - naufil_target)**2).sum()
    naufil_loss.backward()
    naufil_optimizer.step()
    naufil_loss = naufil_loss.get().data
    naufil_loss

    faizan_optimizer.zero_grad()
    faizan_pred = faizan_model(faizan_data)
    faizan_loss = ((faizan_pred - faizan_target)**2).sum()
    faizan_loss.backward()
    faizan_optimizer.step()
    faizan_loss = faizan_loss.get().data
    faizan_loss
  naufil_model.move(trusted_aggregator)
  faizan_model.move(trusted_aggregator)

  with th.no_grad():
    model.weight.set_(((naufil_model.weight.data + faizan_model.weight.data)/2).get())
    model.bias.set_(((naufil_model.bias.data + faizan_model.bias.data)/2).get())
  trusted_aggregator.clear_objects()
  print("naufil's loss: " + str(naufil_loss)+", faizan's loss: " + str(faizan_loss))

naufil's loss: tensor(0.0366), faizan's loss: tensor(0.0211)
naufil's loss: tensor(0.0115), faizan's loss: tensor(0.0021)
naufil's loss: tensor(0.0033), faizan's loss: tensor(3.2985e-07)
naufil's loss: tensor(0.0011), faizan's loss: tensor(0.0003)
naufil's loss: tensor(0.0004), faizan's loss: tensor(0.0005)
naufil's loss: tensor(0.0002), faizan's loss: tensor(0.0006)
naufil's loss: tensor(0.0001), faizan's loss: tensor(0.0005)
naufil's loss: tensor(8.3931e-05), faizan's loss: tensor(0.0004)
naufil's loss: tensor(5.8470e-05), faizan's loss: tensor(0.0003)
naufil's loss: tensor(4.2241e-05), faizan's loss: tensor(0.0002)
naufil's loss: tensor(3.1116e-05), faizan's loss: tensor(0.0002)
naufil's loss: tensor(2.3158e-05), faizan's loss: tensor(0.0001)
naufil's loss: tensor(1.7326e-05), faizan's loss: tensor(0.0001)
naufil's loss: tensor(1.2999e-05), faizan's loss: tensor(7.7089e-05)
naufil's loss: tensor(9.7669e-06), faizan's loss: tensor(5.8043e-05)
naufil's loss: tensor(7.3429e-06), faizan

In [15]:
model(th.tensor([0.,1],requires_grad=True))

tensor([0.9774], grad_fn=<AddBackward0>)